In [5]:
import requests
import pandas as pd
from zipfile import ZipFile
from io import BytesIO
import datetime
import logging

headers = {
    "accept": "application/json",
}

session = requests.Session()

def get_jobs(year, month):
    url = f"https://apidatalake.tesouro.gov.br/ords/custos/tt/demais?organizacao=235876&ano={str(year)}&mes={str(month)}"
    logging.info(f"Starting request for endpoint {url}")
    first_page = session.get(url)
    first_page_content = first_page.json()
    limit = first_page_content["limit"]
    yield first_page_content
    
    next_page_content = first_page_content
    while next_page_content["hasMore"] == True:
        limit += next_page_content["count"]
        next_page_url = url + "&offset=" + str(limit)
        next_page = session.get(next_page_url)
        next_page_content = next_page.json()
        
        if next_page.status_code != 200:
            break
        yield next_page_content
        
data = []
for page in get_jobs(year=2021, month=11):
    data += page["items"]
    
import json
with open('./batch/data_202112.json', 'w') as f:
    json.dump(data, f)
        
    